In [1]:
from typing import List, Optional
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoModel, AutoTokenizer


class ChatGLMService(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    history = []
    tokenizer: object = None
    model: object = None

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    def _call(self,
              prompt: str,
              stop: Optional[List[str]] = None) -> str:
        response, _ = self.model.chat(
            self.tokenizer,
            prompt,
            history=self.history,
            max_length=self.max_token,
            temperature=self.temperature,
        )
        if stop is not None:
            response = enforce_stop_tokens(response, stop)
        self.history = self.history + [[None, response]]
        return response

    def load_model(self, model_name_or_path: str = "THUDM/chatglm-6b"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
        print('加载chatglm-6b', model_name_or_path)
        # 这里使用的是 量化后的模型int4
        self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half().cuda()
        #.half().quantize(4).cuda()
        #.half().cuda()
        self.model=self.model.eval()

In [2]:
from langchain import HuggingFaceHub, LLMChain
from langchain import PromptTemplate

template = """请回答用户提出的问题，如果不知道就回复不知道。问题: 
{question}

回答: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)


llm = ChatGLMService()
llm.load_model()

加载chatglm-6b THUDM/chatglm-6b


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm
)

In [4]:
print(llm_chain.run('你好，请你说一下流程挖掘和任务挖掘的区别'))

The dtype of attention mask (torch.int64) is not bool


流程挖掘和任务挖掘是两种不同的数据挖掘技术，其主要区别在于它们用于分析的数据类型和目标不同。

流程挖掘主要关注业务流程和流程中各个环节之间的关系，旨在发现流程中的潜在问题和优化机会，例如流程效率、成本、质量等方面。流程挖掘通常使用事件驱动的方法，通过分析业务流程中的事件和事件之间的关系，来识别流程中的模式和规律。

任务挖掘则主要关注任务执行过程中的数据收集和分析，旨在发现任务执行中的问题和优化机会，例如任务执行效率、任务完成质量等方面。任务挖掘通常使用任务驱动的方法，通过分析任务执行过程中的数据，来识别任务执行中的模式和规律，并利用这些规律来优化任务的执行效率、完成质量和成本等方面。

因此，流程挖掘和任务挖掘的区别在于它们用于分析的数据类型和目标不同，前者关注业务流程，后者关注任务执行过程。


In [5]:
print(llm_chain.run("你知道艺赛旗和望繁信的产品有什么不一样吗？他们都是做流程挖掘的。"))

很抱歉，我不确定艺赛旗和望繁信的产品是什么，因为他们只是听说过这两个公司，并没有具体了解他们的产品。如果您能提供更多的信息，我将尽力回答您的问题。


In [6]:
question = """
jack Rae在报告中强调LLM的这种数据压缩能力是无损的，并反驳了著名科幻小说作家特德姜在年初时候提出的影响很大的“ChatGPT是对互联网数据的有损压缩”观点。其实吧，你要仔细思考一下，会发现这种LLM是对数据的“无损压缩”的观点是有点水分的。如果我们更严谨地来看，会发现尽管LLM训练过程可以看成是对数据的无损压缩，但是能够达成“无损”的效果，并不单单靠LLM，其实是“LLM+算术编码”一起完成的。

如果LLM通过学习达到足够强的智能程度，能够保证NTP预测后续文字序列的loss是0，也就是说可以根据上文Context，完全精准预测后续Next Token，此时算术编码就不需要了，仅靠LLM就能完整地对数据进行压缩和解码。如果是这种情况，我们说LLM的训练过程或者说LLM在经过训练后，能对数据进行“无损压缩”，并没有任何问题。但是，这是理想情况，目前LLM能做到这点吗？肯定是做不到的，所以LLM给出的后续Next Token预测肯定会有错误，这些被预测错误的Token，其实就代表了LLM压缩数据的信息损失，这种损失是靠算术编码额外对信息进行编码来进行补偿，才达成数据的“无损压缩”效果。所以，更精确的说法，看似应该是这样：

数据无损压缩=LLM模型的有损数据压缩能力+算术编码的编码补偿能力



如果综合下现有的研究结论，我觉得可以大致得出这么一个GPT知识提取的轮廓：当训练好GPT模型后输入Prompt，对于Transformer某个位置对应的输入单词，随着Transformer 不断往上走，GPT通过Attention，把这个单词上文中与自己有关的信息集成到自己的Embedding里，而每层的FFN对当前单词Embedding做变换增加信息，以此方式来不断触发FFN里存储的知识并逐层Refine单词对应的Embedding（类似上面例子里单词“music”的过程）。

Transformer的last token位置也是如此，它的特殊之处在于，从底层到上层，首先会把整个输入上文中最关键的信息，通过Attention拷贝到自己的位置，之后通过这个关键信息来逐步过滤出上文中比较重要的信息。在这个位置的Transformer底层，应该有很多候选答案可供输出，正确答案排名并不靠前，随着Transformer往上走，正确答案排名越来越靠前，而且可以和正确答案竞争的候选答案越来越少，体现为分配给正确答案的概率分布得分越来越高，直到last token的最高层，GPT可以输出正确答案（类似上面例子中单词“by”的过程）。

在训练好的GPT模型中，Transformer底层编码了大量的具体特征或知识点，比如n-gram特征、句法特征等，编码方式采用上述的由多个“多语义神经元”组成的superposition模式；而随着Transformer层数加深，具体知识点逐渐减少，抽象知识点（比如“法语”／“质数”等）逐渐增加，且抽象知识点一般由“单语义神经元”独立编码，且随着Transformer层数越高，编码的特征越抽象。换句话说，Transformer对特征或知识点的编码，存在由低到高越来越抽象的一个知识抽象过程，这一现象在OpenAI最新发布的文章“Language models can explain neurons in language models”也有提及。

另外，“Polysemanticity and Capacity in Neural Networks”这个文章指出了：在模型学习过程中，为了增加模型参数的利用效率，“单语义神经元”会被分配给重要特征，“多语义神经元”会分配给不太重要的特征，而对更加不重要的特征，则模型完全不进行编码。而所谓的“重要性”，指的是对训练loss的影响来说的，也就是说：“单语义神经元”对NTP训练时降低loss影响比较大。这说明对特征或知识点进行抽象，是NTP本身为了快速降低Loss的一种内在驱动力，而这很可能是GPT模型通过Next Token Prediction任务产生智能的关键之一。



OpenAI首席科学家Ilya Sutskever在访谈中曾说： “我们训练LSTM来预测亚马逊评论的下一个字符(NTP)时发现，如果你预测下一个字符足够好，LSTM就会有一个与情绪对应的神经元。这就很好地展示了无监督学习的效果，也验证了下一个字符预测的想法。这个发现对我们的影响很大。”我理解这里说的在网络中出现了与情绪对应的神经元，大概是通过NTP训练任务，在模型内部形成了一个情感判断的知识回路。这个发现（可参考：Learning to Generate Reviews and Discovering Sentiment），确实是后来推动OpenAI把LSTM换成更大规模的Transformer，并在更多数据上采用NTP来进行预训练的重要启发因素。



首先，我们把已知研究结论汇总一下，形成一个整体印象。在本文里，我把某个特征或知识统一称为知识点，因为单使用传统的“特征”很难涵盖某些内容，具体知识点包括语言知识点（n-gram、词法、句法、语义等）、context知识点（比如“输入为法语”这种）、世界知识相关知识点（实体-属性、常识、事件等）以及简单函数回路知识点这几种，它们是细粒度的，我们将其统称为知识点。
综合上述内容可看出，GPT模型通过NTP任务从数据中学习知识，在模型内部建立起两类知识体系：层级化的知识结构以及各种任务回路（参考上图），任务回路是在层级知识体系结构上建立起来的，是用于解决某个任务的、由知识点相互激发形成的固定通路。



根据目前已有研究结论，如果思考大LLM模型和小LLM模型的差异，我们大致可做如下推断：小LLM模型建立了一个粗粒度的、模糊的世界图像，而随着模型规模越来越大，大LLM模型建立起能表征更多细节信息的清晰度越来越高的世界图像。

由上文所述可知，LLM模型的表征能力主要体现在两个方面：从具体到抽象的层级知识结构，以及可以解决很多问题的任务回路。我们从这两方面分头看下大小模型的差异。

Voyager | An Open-Ended Embodied Agent with Large Language Models (minedojo.org)

Voyager continuously improves itself by writing, refining, committing, and retrieving *code* from a skill library. GPT-4 unlocks a new paradigm: “training” is code execution rather than gradient descent. “Trained model” is a codebase of skills that Voyager iteratively composes, rather than matrices of floats. We are pushing no-gradient architecture to its limit. Voyager rapidly becomes a seasoned explorer. In Minecraft, it obtains 3.3× more unique items, travels 2.3× longer distances, and unlocks key tech tree milestones up to 15.3× faster than prior methods.

世界的参数倒影：为何GPT通过Next Token Prediction可以产生智能 - 知乎 (zhihu.com)

按照23年1月份版本的我当时对LLM的理解，把NTP产生智能归纳为”通过NTP任务，LLM在模型参数里学到了一个隐形的知识图谱，当输入Prompt的时候，Prompt包含的概念启动知识图谱相关节点，然后在知识图谱上按照 <激活-扩散> 理论引发知识间的激活扩散和信息传递，由此导致LLM产生了智能”。

OpenAI首席科学家Ilya Sutskever在今年早期的一些公开访谈初步透漏了这个思路。 而后续OpenAI负责大模型团队的 Jack Rae 在 Stanford MLSys 研讨会上做了以“Compression for AGI”为主题的报告，则是从理论层面，概念性地论证了这一理念。

本部分主要参照Jack Rae报告内容，转述OpenAI坚信的“压缩即智能”论证过程。

这两年最大的收获，我感觉其实就是消除了焦虑，对，可以用消除这个词。在去年的时候还经常因为一些技术问题睡不着觉，还会发脾气什么的。然后今年，尤其是手骨折之后，就感觉好像没有什么人什么事，是值得反复考虑的。还有个因素是GPT-4，它也让我意识到所谓的思考其实是没什么意义的，连自娱自乐都效果有限。获得这种能力有一个很大的原因就是，我操作了两个长期、复杂的项目，一个就是流程挖掘，另外一个就是留学移民。当你操作复杂项目的时间，焦点肯定是集中于最重要的决策上，小的细节和人等因素自然变得不重要了。所以，并不是说聚焦的核心是有很多事情，怎么把其他事情过滤或者排优先级。这是很明确的，不至于说你一般的思维解决不了，真正的聚焦的意思，是说你一开始就瞄准比较大、比较重要的事情，那就可以长期一个目标，这样子的优先级才有意义。如果只是一个很近期的很具体的目标。比如说一个月或者一两周就完成的事情，简单的任务拆解一下，先后次序就很清楚了，再怎么调整也锻炼不出来特别的能力。要锻炼能力就必须去解决长期复杂的问题，只有这样的情况下聚焦才有意义，优先级才有意义。只有做这种事情，然后对应的能力才能培养出来。要解决的问题觉得了能力的高低，而不是相反。

我可以先建一个文档阅读器，这个文档阅读器能够联网，除了阅读本地的文件，还可以上网上查询、下载、再阅读。如果搜索到的相关解释不够多，那就在网上再找一些相关的文件下载下来，再继续重新提问、阅读。其次呢，还可以找到论文对应的代码，使用类似于cody的实现方式，把对应的代码也连起来，这样子就解决了阅读代码所有的问题。


我意识到一个问题就是，如果说代码能够教会大模型逻辑推理能力，那么是不是可以用类似生成代码的方式让模型获得更多的逻辑推理能力呢？为什么代码让模型具备推理能力，是因为代码是要靠机器执行的，那么它执行过程中的每一步，都清晰的描述了出来，所以这里边不存在任何模糊的地方，不确定的地方，或者是出错的地方，但是人的逻辑，或者说语言表达出来的逻辑，他没有这么严格的一个推理限定，所以，你让模型学文本时，它就会学会人看起来符合逻辑但实际上里边充满着比较多不太合适，或者是错误的推理判断。那么如何生成这样子的数据呢？很简单，就是类似于代码的生成方式，给一些任务让机器去执行，把执行的过程完整记录下来，然后作为一个训练数据，因为肯定要达到预期的目的，执行过程就是需要有精确的逻辑推理的。

具体的来说，就比如说我有一个实体的机器人，给他一些命令，然后他去做这个事情，把做的过程的每一步生成的日志记录下来。为了保证执行过程中有一个外部的各种噪音，他要做决策选择，那么这种情况下他就需要有一个不确定性的执行环节，而不是每一次执行过程都完全一样的。在外界变化的时间才能考验他的决策，然后这个样子才能注入新的知识

我认为语言的功能，是进行信息压缩。信息压缩的作用，则是让原本只能处理少量信息的神经中枢，能透过信息压缩机制，近乎无限的扩张其信息处理能力，即增强“人脑算力”。

语言是一种工具，一种通过实现信息压缩，增强人脑信息处理能力，提高人脑信息处理总量，即“算力”的工具。 工具与工具之间有优劣之别，使用工具的人在技巧上也有差异。 理解能力，就是接收、解压方面的技巧。 表达能力，就是发送、压缩方面的技巧。 技巧是需要在实战中磨练的。没有频繁的沟通，就没有好的语言使用技巧。不论阅读还是演说，都需要练。 而如果你有意识地进行这些方面的练习，最终会促成你大脑“算力”的增强。

我解释了为什么我认为语言是一种用于实现信息压缩功能的工具，也解释了在我眼中，人脑这个从原始设计（演化）看并不能处理海量信息的系统，如今通过语言工具的杠杆作用，能处理海量信息的原因。

对语言中呈现的信息压缩现象，其实可以做更多的研究和分析，不管是比较不同语言的信息压缩能力和解压缩时的无损性，还是研究人脑在信息压缩和解压过程中的活动方式，都会是很有趣的题目。 但对我来说，我的兴趣会更集中于语言中的“信息压缩包”，或者说“信息结晶”，在内容被统治者强力篡改时，引发的社会现象。

有些具备更丰富相关知识的人能从同一个词汇中解读出更多信息，从同一首诗中（因为对作者生平和时代背景的了解）解读出更多含义。这些“额外”的内容，就不算是包装中的标准配置，而象有美食追求的泡面者在碗中另加的青菜、鸡蛋一样，是“私人藏品”了。 这种在拆开方便面后，因烹饪技巧的不同，额外加料的不同产生的味道差异，多数情况下不影响交流。毕竟螺丝粉不会被煮成热干面。 但当词汇的含义被篡改呢？ 这就等于在方便面里掺毒。

The New Yorker on Twitter: "Can large-language models help humans with the creation of original writing? To answer that, we need to be specific about what we mean by that question. https://t.co/ugAZOOfZpa" / Twitter

chatgpt并不特别引发Acc圈子的关注，特德姜已经直接点名其在信息转换的过程中，原始信息压缩、失真、破碎的情况。chatgpt即使是当下公众视野中已知的最先进的机械，仍然不能脱离人类学习过程中必然遇到的麻烦，没有必要对现在的它抱太多期望甚至赋予神格。

大模型技术体系是推动新范式发展的动力引擎。其能力主要体现在信息压缩、泛化能力、推理能力、对齐能力、利用算力等方面。 自然语言处理是突破的关键,它为建立世界模型提供基础。

请总结这一段话
"""

len(question)

5918

In [7]:
for i in range(10):
    print(llm_chain.run(question))

Token indices sequence length is longer than the specified maximum sequence length for this model (2953 > 2048). Running this sequence through the model will result in indexing errors


GPT模型通过NTP任务学习到隐形的知识图谱，并在知识图谱上引发知识间的激活扩散和信息传递，从而生成智能。在知识图谱中，语言知识点和 context 知识点是细粒度的知识，而 context 知识点是由多个语言知识点和 context 组成的复杂知识点。GPT模型通过训练，学习到这些知识，并通过知识图谱中的节点和边，将知识进行组织，生成智能。
GPT模型的压缩能力是指其能够对数据进行压缩，使得数据在存储和传输时更加高效。这种能力对于自然语言处理非常重要，因为自然语言处理需要大量的数据来进行训练，而GPT模型的压缩能力能够有效地减少数据量，提高训练效率。此外，GPT模型的压缩能力也能够使得数据更加易于存储和传输，从而提高数据的可用性和可访问性。
GPT模型通过NTP任务学习到隐形的知识图谱，并在知识图谱中引发知识间的激活扩散和信息传递，从而生成智能。在知识图谱中，语言知识点和 context 知识点是细粒度的知识，而 context 知识点是由多个语言知识点和 context 组成的。GPT模型通过将输入的文本与知识图谱中的相应知识点相连接，并通过注意力机制来获取上下文信息，从而生成文本。

在 GPT 模型中，知识图谱的层级结构包括语言知识点、context 知识点和任务回路。任务回路是用于解决特定问题的通路，由多个语言知识点和 context 知识点组成。GPT模型通过将输入的文本与知识图谱中的相应知识点相连接，并通过注意力机制来获取上下文信息，从而生成文本。

在 GPT 模型中，知识图谱的编码方式采用由多个“多语义神经元”组成的序列编码方式。多语义神经元是指能够同时编码多个语言知识点和 context 知识点的神经元。GPT模型通过将输入的文本与知识图谱中的相应知识点相连接，并通过注意力机制来获取上下文信息，从而生成文本。

在 GPT 模型中，知识图谱的表征能力主要体现在从具体到抽象的层级知识结构和可以解决很多问题的任务回路上。GPT模型通过将输入的文本与知识图谱中的相应知识点相连接，并通过注意力机制来获取上下文信息，从而生成文本。知识图谱的表征能力主要体现在从具体到抽象的层级知识结构和可以解决很多问题的任务回路上。
GPT模型的压缩能力是指其能够对文本进行压缩，使得文本在存储和传输时更加高效。这种能力对于自然语言处理非常重要，因为自然语言文本通常非常大，而

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_300648/3368835442.py:2 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_300648/3368835442.py'                       │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/base.py:236 in run      │
│                                                                                                  │
│   233 │   │   if args and not kwargs:                                                            │
│   234 │   │   │   if len(args) != 1:                                                             │
│   235 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   237 │   │                                                                                      │
│   238 │   │   if kwargs and not args:                                                            │
│   239 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/base.py:140 in __call__ │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/base.py:134 in __call__ │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /home/jinzy/.conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/llm.py:69 in _call      │
│                                                                                                  │
│    66 │   │   inputs: Dict[str, Any],                                                            │
│    67 │   │   run_manager: Optional[CallbackManagerForChainRun] = None,                          │
│    68 │   ) -> Dict[str, str]:                             